In [ ]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

Dependencies
---

This notebook uses automatic equation numbering. This requires a mathjax extension to be installed on the Jupyter server:
```
conda install -c conda-forge jupyter_contrib_nbextensions 
jupyter contrib nbextension install --user
jupyter nbextension enable equation-numbering/main
```
Then uncheck the general disabling of configuration of extensions in the Jupyter notebook Editor menu, Edit->nbextensionsconfig 
see: https://stackoverflow.com/questions/41241984/equation-numbering-in-jupyter-notebooks


Packages
--
math
numpy
matplotlib
spharm



Primitive Equations, in vorticity & divergence form and $\sigma$-coordinate
---

<u>Prognostic Equations:</u>

$$
\frac{\partial \zeta}{\partial t} = - \nabla\circ{ \left[ (\zeta + f) \vec u  \right]} - \vec k \cdot \vec\nabla \times {\left[ \dot{\sigma} \frac{\partial{\vec u}}{\partial{\sigma}} + RT \vec\nabla ln(p_s)\right]} \label{eq: vorticity_trend} \tag{1}
$$

$$
\frac{\partial D}{\partial t} = \vec k \cdot \vec\nabla\times{ \left[ (\zeta + f) \vec u  \right]} -  \vec\nabla \circ {\left[ \dot{\sigma} \frac{\partial{\vec u}}{\partial{\sigma}} + RT \vec\nabla ln(p_s)\right]} + \nabla^2 {\left[ \Phi + \frac{\vec u \cdot \vec u}{2} \right]}\label{eq: divergence_trend}  \tag{2} 
$$

$$
\frac{\partial T}{\partial t} = k \frac{T \omega}{\sigma p_s} + \frac{Q}{c_p} - \vec u \cdot \vec\nabla T - \dot\sigma \frac{\partial T}{\partial \sigma} \label{eq:temperature_trend} \tag{3}
$$


$$
\frac{\partial p_s}{\partial t} = - \int_{\sigma=0}^{\sigma=1}{\nabla\circ\left( p_s \vec u \right)d\sigma} \label{eq:surface_pressure_trend} \tag{4}
$$

<u>Diagnostic Equations:</u>

$$
\omega(\sigma) = \sigma \vec u \cdot \vec\nabla p_s - \int_0^{\sigma}{\nabla \circ \left( p_s \vec u \right) d\sigma} \label{eq:vertical_speed} \tag{5}
$$

$$
\dot\sigma(\sigma) = -\frac{1}{p_s} \left[  \sigma \frac{\partial p_s}{\partial t} + \int_0^{\sigma}{\nabla \circ \left( p_s \vec u \right) d\sigma} \right] \label{eq:sigma_dot} \tag{6}
$$

$$
\Phi(\sigma) = g.z_0 - \int_{ln(\sigma)=0}^{ln(\sigma)}{RTd(ln\sigma)} \label{eq:gravitational_potential} \tag{7}
$$

The change in vertical vorticity $\zeta$ is given by equation $\eqref{eq: vorticity_trend}$

The change in horizontal divergence $D$ is given by equation $\eqref{eq: divergence_trend}$


In [ ]:
def plot(X, title='', log=False):
    
    if log:
        temp = np.exp(X)
    else:
        temp=X
    
    plt.figure(figsize=(12,6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_global()
    ax.gridlines()
    im=plt.contourf(lons1d*180/np.pi, lats1d*180.0/np.pi, temp, transform=ccrs.PlateCarree(), cmap=plt.cm.Spectral)
    plt.colorbar(im, format='%.0e')
    #plt.colorbar(im, format='%.2f')
    plt.title(title)
    plt.show()
    
def splot(sX, title='', log=False):
    X = sp.spectogrd(sX)
    plot(X, title, log)
        


In [ ]:
#%matplotlib notebook

import math 
import numpy as np
import matplotlib.pyplot as plt
from spharm import Spharmt, getspecindx, gaussian_lats_wts

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import time

import numpy.random as npran
import matplotlib.animation as animation

#------
#CONFIG
#------

K = 1.380649e-23 #J/K
R = 287.         #J/kg/K
Cp = 1004.       #J/kg/K
Cv = 717.        #J/kg/K

rsphere = 6.37122e6 #m earth radius
omega   = 7.292e-5  #rad/s rotation rate

itmax = 100 #number of iteration steps
dt = 0.1*3600  #timestep in seconds

numlevels = 2 #number of atmospheric layers
efold = 3600.  #for hyperdiffusion
ndiss = 8 #for hyperdiffusion

nlon = 128 #longitude grid
ntrunc = 42 #number of spherical harmonics (triangular truncation)
delta = 2.*np.pi/nlon

gridtype = 'regular'
nlat = int(nlon/2)+1
lats1d = 0.5*np.pi-delta*np.arange(nlat)
wts = np.cos(lats1d)
#gridtype = 'gaussian'
#nlat = int(self.nlon/2)
#lats1d, wts = gaussian_lats_wts(nlat)
#lats1d = lats1d*np.pi/180. # convert to radians.
lons1d = np.arange(-np.pi,np.pi,delta)
lons,lats = np.meshgrid(lons1d,lats1d)
f = 2.*omega*np.sin(lats)  #horizontal coriolis force

sp = Spharmt(nlon,nlat,rsphere=rsphere,gridtype=gridtype)

# create laplacian operator and its inverse.
indxm, indxn = getspecindx(ntrunc)
indxn = indxn.astype(np.float32)
totwavenum = indxn*(indxn+1.0)
lap = -totwavenum/sp.rsphere**2
ilap = np.zeros(lap.shape, np.float32)
ilap[1:] = 1./lap[1:]

# hyperdiffusion operator
hyperdiff = -(1./efold)*(totwavenum/totwavenum[-1])**(ndiss/2)


#----
#INIT
#----

#sigma values at the middle of the layers
sigma = np.ones( numlevels, np.float32)
dsigma = 1./numlevels
for i in range(numlevels):
    sigma[i] = (i+0.5)*dsigma

#initial horizontal wind field (u,v) in m/s on the (lat,lon) grid at the middle of each atmospheric layer
u = np.zeros( (nlat, nlon, numlevels), np.float32)
v = np.zeros( (nlat, nlon, numlevels), np.float32)

#zonal flow
u[:,:,0] = 50*np.sin(np.pi*np.sin(lats)**2)

#initial spectral vorticity and divergence
sVort, sD = sp.getvrtdivspec(u, v, ntrunc)

#add noise to initial vorticity
psipert = np.zeros((nlat,nlon),np.float32)
psipert[:,:] = 5.e6*np.sin((lons-np.pi))**12*np.sin(2.*lats)**12
psipert = np.where(lons[:,:] > 0., 0, psipert)
sVort[:,0] += lap * sp.grdtospec(psipert,ntrunc)
  
    
#initial temperature field on the grid and its spectral representation, at the middle on each atmosphere layer
T = (273.+15) * np.ones( (nlat, nlon, numlevels), np.float32)
sT = sp.grdtospec(T, ntrunc)

#initial logarithm of surface pressure field on the grid and its spectral representation
LnPs = np.log(100e3) * np.ones( (nlat, nlon), np.float32 )
sLnPs = sp.grdtospec(LnPs, ntrunc)

plot(u[:,:,0], title='u[0]')

#plot(np.exp(LnPs))
#plot(np.exp(sp.spectogrd(sLnPs)))
#plot(T[:,:,0])
#plot(sp.spectogrd(sT)[:,:,0])

#------
#TRENDS
#------

def vertdiff(X):
    dXds    = np.zeros_like(X, np.float32)
    
    dXds[:,:,0] = (X[:,:,1] - X[:,:,0])/dsigma
    dXds[:,:,numlevels-1] = (X[:,:,numlevels-1] - X[:,:,numlevels-2])/dsigma
    
    for i in range(1, numlevels-1):
        dXds[:,:,i] = (X[:,:,i+1] - X[:,:,i-1])/2./dsigma
    
    return dXds

#calculate the trend of prognostic variables given their current value according to the primitive equations
#input params in spectral form    
def dydt(sVort, sD, sT, sLnPs):

    Phi      = np.zeros((nlat, nlon, numlevels), np.float32)
    G        = np.zeros((nlat, nlon, numlevels), np.float32)
    intG     = np.zeros((nlat, nlon, numlevels), np.float32)
    dLnPsdt  = np.zeros((nlat, nlon), np.float32)  #trend of dlnPs/dt, on the grid
    sigmadot = np.zeros((nlat, nlon, numlevels), np.float32)
    dTdt     = np.zeros((nlat, nlon, numlevels), np.float32)
    dTds     = np.zeros((nlat, nlon, numlevels), np.float32)
    dsVortdt = np.full_like(sVort, 0.)
    dsDdt    = np.full_like(sD, 0.)

    T                = sp.spectogrd(sT)
    dTdx, dTdy       = sp.getgrad(sT)   #gradient on the grid
    u, v             = sp.getuv(sVort, sD)
    dLnPsdx, dLnPsdy = sp.getgrad(sLnPs)   #gradient of logarithm of surface pressure, on the grid; sPs is initialized as log of surface pressure
    Vort             = sp.spectogrd(sVort)
    D                = sp.spectogrd(sD)

    #plot(u[:,:,0], title='u[{i}]'.format(i=0))
    
    
    #trend for lnPs, first calculated on the grid and brought to spectral form
    #-------------------------------

    #G = div(u) + u.grad(lnPs)
    #a temporary value used in several calculations
    #plot(dLnPsdx, title='dLnPsdx')
    #plot(dLnPsdy, title='dLnPsdy')
    for i in range(0, numlevels):
        G[:,:,i] = D[:,:,i]    #u[:,:,i] + v[:,:,i]
        G[:,:,i] += u[:,:,i]*dLnPsdx + v[:,:,i]*dLnPsdy
        #plot(G[:,:,i], title='G[{i}]'.format(i=i))
        #plot(u[:,:,i], title='u[{i}]'.format(i=i))
        #plot(v[:,:,i], title='v[{i}]'.format(i=i))
        #plot(D[:,:,i] - u[:,:,i] - v[:,:,i], title='D-(u+v)[{i}]'.format(i=i))
        
        intG[:,:,i] = G[:,:,i]*dsigma
        if i>0:
            intG[:,:,i] += intG[:,:,i-1]
    
    dLnPsdt = -intG[:,:,numlevels-1] 
    dsLnPsdt = sp.grdtospec(dLnPsdt, ntrunc)
    #plot(dLnPsdt, title='dLnPsdt', log=False)

    
    #temperature trend
    #-----------------

    #sigmadot on the (lat,lon) grid at the middle of each atmosphere level)
    for i in range(0, numlevels):
        sigmadot[:,:,i] = - sigma[i] * dLnPsdt - intG[:,:,i]
    
    #dT/sigma on the grid
    dTds = vertdiff(T)
    for i in range(0, numlevels):
        #plot(dTdt[:,:,i])
        #plot(dTds[:,:,i])
        #plot(dLnPsdy[:,:])

        dTdt[:,:,i] = u[:,:,i]*dLnPsdx + v[:,:,i]*dLnPsdy
        dTdt[:,:,i] -= intG[:,:,i] / sigma[i]
        dTdt[:,:,i] *= K*T[:,:,i]
        #plot(R*T[:,:,i], title='RT[{i}]'.format(i=i))
        #plot(dTdt[:,:,i], title='dTdt[{i}]'.format(i=i))
        
        #dTdt[:,:,i] += Q[:,:,i]/Cp

        dTdt[:,:,i] -= dTdx[:,:,i]*u[:,:,i] + dTdy[:,:,i]*v[:,:,i]
       
        dTdt[:,:,i] -= sigmadot[:,:,i] * dTds[:,:,i]
      
    dsTdt = sp.grdtospec(dTdt, ntrunc)
      
    #vorticity and divergence trends
    #-------------------------------
        
    #Phi on the (lat,lon) grid at the middle of each atmosphere level)
    for i in range(0, numlevels):
        Phi[:,:,i] = -R*T[:,:,i] * np.log(dsigma)
        if i>0:
            Phi[:,:,i] += Phi[:,:,i-1]
        #plot(Phi[:,:,i], title="Phi[{i}]".format(i=i))
        
    duds = vertdiff(u)
    dvds = vertdiff(v)
    for i in range(0, numlevels):
        #A=(zeta+f)*u
        sRotA, sDivA = sp.getvrtdivspec( (Vort[:,:,i]+f)*u[:,:,i], (Vort[:,:,i]+f)*v[:,:,i], ntrunc)   

        #B = sigmadot*dudsigma + RT*grad[ln(ps)]
        Bx = sigmadot[:,:,i] * duds[:,:,i] + R*T[:,:,i] * dLnPsdx
        By = sigmadot[:,:,i] * dvds[:,:,i] + R*T[:,:,i] * dLnPsdy

        sRotB, sDivB = sp.getvrtdivspec(Bx, By, ntrunc)

        dsVortdt[:,i] = -sDivA - sRotB
        #splot(dsVortdt[:,i])

        dsDdt[:,i] = sRotA - sDivB
        
        sKE = sp.grdtospec(0.5*u[:,:,i]**2+v[:,:,i]**2, ntrunc)
        sPhi = sp.grdtospec(Phi[:,:,i], ntrunc)
        dsDdt[:,i] += lap * (sPhi+sKE)
        #splot(dsDdt[:,i])

    '''
    for i in range(0, numlevels):
        dsVortdt[:,i] += hyperdiff * sVort[:,i]    
        dsDdt[:,i]   += hyperdiff * dsDdt[:,i]    
        dsTdt[:,i]    += hyperdiff * dsTdt[:,i]    
    dsLnPsdt += hyperdiff * dsLnPsdt
    '''
    
    return dsVortdt, dsDdt, dsTdt, dsLnPsdt
      
#---------
#INTEGRATE
#---------

#the function returns the next value with the RK4 method
#y is a vector; 
def stepforward_rk4(sVort, sD, sT, sLnPs):
    dsVort1, dsD1, dsT1, dsLnPs1 = dydt(sVort, sD, sT, sLnPs)
    dsVort2, dsD2, dsT2, dsLnPs2 = dydt(sVort+0.5*dsVort1*dt, sD+0.5*dsD1*dt, sT+0.5*dsT1*dt, sLnPs+0.5*dsLnPs1*dt)
    dsVort3, dsD3, dsT3, dsLnPs3 = dydt(sVort+0.5*dsVort2*dt, sD+0.5*dsD2*dt, sT+0.5*dsT2*dt, sLnPs+0.5*dsLnPs2*dt)
    dsVort4, dsD4, dsT4, dsLnPs4 = dydt(sVort+dsVort2*dt, sD+dsD2*dt, sT+dsT2*dt, sLnPs+dsLnPs2*dt)
    return sVort + dt*(dsVort1+2*dsVort2+2*dsVort3+dsVort4)/6, \
           sD + dt*(dsD1+2*dsD2+2*dsD3+dsD4)/6, \
           sT + dt*(dsT1+2*dsT2+2*dsT3+dsT4)/6, \
           sLnPs + dt*(dsLnPs1+2*dsLnPs2+2*dsLnPs3+dsLnPs4)/6

def stepforward(sVort, sD, sT, sLnPs):
    dsVort1, dsD1, dsT1, dsLnPs1 = dydt(sVort, sD, sT, sLnPs)
    #splot(dsVort1[:,0])
    #splot(dsD1[:,0])
    #splot(dsT1[:,0])
    #splot(dsLnPs1)
    return sVort + dt*dsVort1, \
           sD + dt*dsD1, \
           sT + dt*dsT1, \
           sLnPs + dt*dsLnPs1


#splot(sVort[:,0], title='vort')
#splot(sD[:,0], title='div')
#splot(sT[:,0], title='T')
#splot(sLnPs, title='Ps', log=True)
for ncycle in range(itmax):
    t = (1+ncycle)*dt
    sVort, sD, sT, sLnPs = stepforward_rk4(sVort, sD, sT, sLnPs)
    if ncycle % 1 == 0:
        splot(sLnPs, title='Ps t={ncycle}'.format(ncycle=ncycle), log=True)
        #splot(sVort[:,0], title='vort t={ncycle}'.format(ncycle=ncycle))
        #splot(sD[:,0], title='div t={ncycle}'.format(ncycle=ncycle))
        #splot(sT[:,0], title='T t={ncycle}'.format(ncycle=ncycle))

        

    